<a href="https://colab.research.google.com/github/fakhrulnurmulyana/VolunterPembuatanAplikasi/blob/main/HukumQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h1>Chatbot HukumQ</h1>**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [ ]:
%%writefile content.json
{"intents":
  [
    {
      "tag" : "sapaan",
      "input" : ["Halo", "Selamat datang", "Apa kabar hari ini?", "Assalamualaikum", "Mari kita berbicara", "Apa kabar, teman?", "Salam kenal",  "Hai, kawan!", "Hai, sobat!",  "Salam sejahtera", "p", ".", "Sampurasun", "baa kaba?", "oi", "oy", "spada"],
      "responses" : ["Halo", "Selamat datang", "Bagaimana keadaanmu?", "Hai, apa kabarmu?", "Apa yang terjadi?", "Apa yang bisa saya bantu?", "Ada yang perlu diobrolkan?", "Apa kabar hari ini?", "Assalamualaikum", "Mari kita berbicara", "Apa kabar, teman?", "Salam kenal", "Hai juga",  "Hai, kawan!", "Hai, sobat!",  "Salam sejahtera", "Halo!", "Baik, terima kasih. Bagaimana denganmu?", "Terima kasih, senang berada di sini.", "Baik, terima kasih. Bagaimana denganmu?", "Hai! Kabarku baik, terima kasih. Bagaimana denganmu?", "Tidak ada yang spesial, bagaimana denganmu?", "Apa yang bisa saya bantu untukmu?", "Tentu, kita bisa obrolkan apa saja.", "Hari ini baik-baik saja, terima kasih. Bagaimana denganmu?", "Waalaikumsalam!", "Tentu, kita bisa berbicara.", "Semua baik, terima kasih. Bagaimana denganmu?", "Hai, teman! Bagaimana kabarmu?", "Tidak ada yang baru, apa kabar?", "Salam kenal juga!", "Hai juga!", "Hai, kawan! Apa kabar?", "Salam sejahtera!", "hai orang asing, apa yang kamu mau?"]
    }
    {
      "tag" : "kabar",
      "input" : [],
      "responses" : []
    }
    {
      "tag" : "siapa",
      "input" : [],
      "responses" : []
    }
    {
      "tag" : "lokasi",
      "input" : [],
      "responses" : []
    }
    {
      "tag": "hukum_umum",
      "patterns": ["Apa itu hukum?", "Bagaimana proses peradilan bekerja?"],
      "responses": ["Hukum adalah...", "Proses peradilan melibatkan..."]
    }
    {
      "tag" : "",
      "input" : [],
      "responses" : []
    }
    {
      "tag" : "",
      "input" : [],
      "responses" : []
    }
    {
      "tag" : "",
      "input" : [],
      "responses" : []
    }
    {
      "tag" : "",
      "input" : [],
      "responses" : []
    }
    {
      "tag" : "",
      "input" : [],
      "responses" : []
    }
    {
      "tag" : "",
      "input" : [],
      "responses" : []
    }
    {
      "tag" : "",
      "input" : [],
      "responses" : []
    }
    {
      "tag" : "pamitan",
      "input" : ["terima kasih", "terimakasih", "terimakasih banyak", "terima kasih", "makasih", "dadah", "adios", "okay", "oke", "sampai bertemu lagi", "Sukses selalu.", "Hatur nuhun.", "Senang berinteraksi denganmu.", "Terima kasih atas bantuanmu.", "Berterima kasih atas dukunganmu.", "Semoga harimu menyenangkan.", "Sampai jumpa lain waktu.", "Terima kasih banyak ya!", "Semoga harimu penuh berkah.", "Sukses untukmu!", "Hingga kita kembali bertemu.", "Terima kasih atas waktu dan perhatiannya.", "Sehat selalu, ya!", "Semoga harimu indah seperti senyummu.", "Sampai jumpa di lain kesempatan.", "Semoga kebaikanmu berlipat ganda.", "Berterima kasih atas kerjasamanya.", "Sukses dan bahagia selalu.", "Semoga kesuksesan selalu menyertaimu.", "Mudah-mudahan hari esok lebih baik lagi.", "Semoga langkahmu selalu diberkahi.", "Terima kasih, semoga harimu menyenangkan.", "Sukses selalu dalam setiap langkahmu.", "Terima kasih atas kebaikanmu.", "Semoga kesuksesan senantiasa menyertaimu.", "Sampai jumpa di kesempatan berikutnya.", "Semoga kita bisa berinteraksi lagi nantinya.", "Terima kasih atas kerja samanya.", "Semoga hari-harimu selalu bahagia.", "Hingga kita bertemu lagi, terima kasih!"],
      "responses" : ["Sama-sama, senang bisa membantu!", "Terima kasih juga atas kerja samanya.", "Sukses selalu untuk Anda juga!", "Semoga kita bisa berinteraksi lagi di lain waktu.", "Sampai jumpa, semoga harimu juga menyenangkan!", "Hatur nuhun, semoga harimu dipenuhi kebahagiaan.", "Terima kasih banyak ya! Semoga sukses terus.", "Senang berinteraksi denganmu juga, semoga harimu baik-baik saja.", "Adios! Hasta luego.", "Semoga langkahmu selalu diberkahi dan sukses.", "Makasih, semoga semua rencanamu berhasil dengan baik.", "Terima kasih atas dukunganmu, semoga kita bisa bertemu lagi.", "Sehat selalu, dan semoga kita bisa berkumpul lagi.", "Semoga harimu indah dan penuh canda tawa.", "Sukses dalam setiap langkahmu, sampai jumpa!", "Terima kasih atas waktu dan perhatiannya, semoga bermanfaat.", "Terima kasih, semoga harimu juga penuh berkah.", "Sampai jumpa di kesempatan berikutnya, semoga bahagia selalu.", "Terima kasih, semoga hari esok membawa kebaikan.", "Hatur nuhun, semoga kita bisa berkolaborasi lagi di masa depan."]
    }
    {
      "tag": "fallback",
      "patterns": ["*"],
      "responses": ["Maaf, saya tidak dapat memahami pertanyaan Anda. Bisakah Anda mencoba pertanyaan lain?"]
    }
  ]
}

**<h2>Mengimport Data</h2>**

In [ ]:
# Mengimport dataset
with open('content.json') as content :
  data1 = json.load(content)

In [ ]:
# Merubah data jadi lists
tags = []
inputs = []
responses = {}
for intent in data1["intents"]:
  responses[intent["tag"]]=intent["responses"]
  for lines in intent["input"]:
    input.append(lines)
    tags.append(intent['tag'])

In [ ]:
data = pd.DataFrame({"inputs":inputs,
                      "tags":tags})

In [ ]:
data

**<h2>Pre-Processing</h2>**

In [ ]:
# Menghilangkan tanda baca
import string
data["inputs"] = data["inputs"].apply(lamda wrd:[ltrs.lower() for ltrs in wrd if ltrs in wrd if ltrs not in string.punctation])
data["inputs"] = data["inputs"].apply(lamda wrd: "".join(wrd))
data

In [ ]:
# Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 2000)
tokenizer.fit_on_text(data["inputs"])
train = tokenizer.text_to_sequences(data["inputs"])
# Apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

# Encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data["tags"])

In [ ]:
input_shape = x_train.shape[1]
print(input_shape)

In [ ]:
# Mendefenisikan Vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ", vocabulary)
output_length = le.classes_shape[0]
print("output length : ", output_length)

**<h2>Train (Neural Network)</h2>**

In [ ]:
# LSTM

#i = Input(shape = (input_shape))
#x = Embedding(vocabulary+1, 10) (i)
#x = LSTM(10, return_sequences=True)(x)
#x = Flatten()(x)
#x = Dense(output_length, activation="softmax")(x)
#model = Model(i,x)

model = Sequential([
    Embedding(vocabulary + 1, 10, input_shape=(input_shape,)),
    LSTM(10, return_sequences=True),
    Flatten(),
    Dense(output_length, activation="softmax")
])

In [ ]:
#compile

model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "adam",
              mentrics = ["accuracy"])

In [ ]:
#Training
#train = model.fit(x_train, y_train, epochs =200)

**<h2>Model Analisis</h2>**

In [ ]:
# Plotting akurasi
plt.plot(train.history["accuracy"], label= "training set accuracy")
plt.plot(train.history["loss"], label= "traiing set loss" )
plt.legend()

**<h2>Testing</h2>**

In [ ]:
#chatting
import random

while True:
  text_p = []
  prediction_input = input("you :")

  # Pre-Processing input
  prediction_input = [letters.lower() for latters in prediction_input if latters not in string.punctation]
  prediction_input = ' '.join(prediction_input)
  text_p.append(pridiction_input)

  # Tokenizer dan Padding input
  prediction_input = tokenizer.text_to_sequences(text_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input], input_shape)

  # Mendapatkan output dari model
  output = model.predict(prediction_input)
  output = output.argmax()

  # Mencari tag yang sesuai
  response_tag = le.inverse_transform([output])[0]
  print("HukumQ :", random.choice(response[response_tag]))
  if response_tag == "goodbye" :
    break


**<h2>kesimpulan</h2>**